In [1]:
# 作業フォルダの移動
%cd C:\Users\tomonari\Documents\SONY_DICE

c:\Users\tomonari\Documents\SONY_DICE


In [2]:
import os
import cv2
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

from mylib import enum_files, make_resize, find_annotation_file, load_annotation, save_annotation, make_yolo_files, move_files_to_folder

In [3]:
def preprocess(src_dir: str, dst_dir: str, img_size: int, override=True) -> None:
    # 画像とアノテーションの出力フォルダを作成
    dst_images_dir = os.path.join(dst_dir, "images")  # 出力画像フォルダ
    dst_labels_dir = os.path.join(dst_dir, "labels")  # 出力アノテーションフォルダ
    os.makedirs(dst_images_dir, exist_ok=True)
    os.makedirs(dst_labels_dir, exist_ok=True)

    # ソースフォルダの処理対象の画像ファイルを列挙
    src_files = enum_files(src_dir, "*/*.jpg")

    # 指定した画像ファイルを変換する処理（マルチスレッドで並列処理させるために関数化）
    def proc_image(src_file: str, override: bool) -> None:
        # override=False の場合、出力フォルダに同名のファイルが存在する場合は処理をスキップ
        dst_img_file = os.path.join(dst_images_dir, os.path.basename(src_file).replace(".jpg", ".png"))
        if not override and os.path.exists(dst_img_file):
            print(f"Skip {src_file}")
            return

        # 画像を読み込んで一連の処理を適用
        img = cv2.imread(src_file)
        img = make_resize(img, resize=img_size)
        # 加工した画像を保存
        cv2.imwrite(dst_img_file, img)
        print(f"Save {src_file} -> {dst_img_file}")

        # アノテーションファイルを読み込み（ファイルが存在する場合）
        ann_file = find_annotation_file(src_file)
        if ann_file:
            # アノテーションファイルの読み込み＆画像のクロップに合わせて座標を変換
            df_ann = load_annotation(ann_file)
            # アノテーションを保存
            dst_ann_file = os.path.join(dst_labels_dir, os.path.basename(ann_file))
            save_annotation(dst_ann_file, df_ann, override=True)

    # 対象画像の処理を実行（マルチスレッドで並列実行）
    # スレッド数は max_workers の値で調整可能
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(proc_image, src_files, [override] * len(src_files))

In [4]:
# 出力画像の解像度と保存先のフォルダの設定
IMG_SIZE = 640
OUT_DIR = f"./data/yolo_{IMG_SIZE}"  # e.g. "./data/yolo_640"

## ノイズ画像を追加

In [13]:
import random
import numpy as np

def gaussnoise(img: np.ndarray):
    #　ノイズ画像を生成する
    mean = 0
    sigma = 96
    sigma = random.uniform(0, sigma)
    noise = np.random.normal(0, sigma, np.shape(img))

    img_noise = img + noise

    img_noise[img_noise > 255] = 255
    img_noise[img_noise < 0] = 0

    return img_noise

In [14]:
import cv2
import os
import shutil
from mylib import enum_files

# ソースフォルダの処理対象の画像ファイルを列挙
src_files = enum_files("./data/src/org", "*/*.jpg")

# 画像を読み込んで一連の処理を適用
for file in src_files:
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    anno_file = os.path.join("./data/src/org/labels", os.path.basename(file).replace(".jpg", ".txt"))
    for i in range(3):
        dst_img_file = os.path.join("./data/src/noise/images", os.path.basename(file).replace(".jpg", f"_gn_{i}.jpg"))
        dst_anno_file = os.path.join("./data/src/noise/labels", os.path.basename(file).replace(".jpg", f"_gn_{i}.txt"))
        img_noise = gaussnoise(img)
        # 加工した画像を保存
        cv2.imwrite(dst_img_file, img_noise)
        # アノテーションファイルをコピーしてリネーム
        if os.path.exists(anno_file):
            shutil.copyfile(anno_file, dst_anno_file)
        print(f"Save {file} -> {dst_img_file}")



Save ./data/src/org\images\train_image100096.jpg -> ./data/src/noise/images\train_image100096_gn_0.jpg
Save ./data/src/org\images\train_image100096.jpg -> ./data/src/noise/images\train_image100096_gn_1.jpg
Save ./data/src/org\images\train_image100096.jpg -> ./data/src/noise/images\train_image100096_gn_2.jpg
Save ./data/src/org\images\train_image100602.jpg -> ./data/src/noise/images\train_image100602_gn_0.jpg
Save ./data/src/org\images\train_image100602.jpg -> ./data/src/noise/images\train_image100602_gn_1.jpg
Save ./data/src/org\images\train_image100602.jpg -> ./data/src/noise/images\train_image100602_gn_2.jpg
Save ./data/src/org\images\train_image100942.jpg -> ./data/src/noise/images\train_image100942_gn_0.jpg
Save ./data/src/org\images\train_image100942.jpg -> ./data/src/noise/images\train_image100942_gn_1.jpg
Save ./data/src/org\images\train_image100942.jpg -> ./data/src/noise/images\train_image100942_gn_2.jpg
Save ./data/src/org\images\train_image101010.jpg -> ./data/src/noise/imag

## アノテーション済み画像データをtrainとvalに分割する

In [19]:
# 既に画像ファイルが存在したときに上書きするかどうかの設定（Trueで上書き, Falseでスキップ）
# 画像の変換処理を変更した場合は、Trueにして既存ファイルを上書きするようにしてください
# Falseを指定すると、既にファイルが存在した場合は処理をスキップします
OVERRIDE = False

# ソースフォルダのtrainデータの2割をvalデータに分割
images = [os.path.join("./data/src/org/images", x) for x in os.listdir("./data/src/org/images")]
labels = [os.path.join("./data/src/org/labels", x) for x in os.listdir("./data/src/org/labels") if x.endswith(".txt")]

images.sort()
labels.sort()

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=123)

# 分割したデータを格納
move_files_to_folder(train_images, "./data/src/train/images")
move_files_to_folder(train_labels, "./data/src/train/labels")
move_files_to_folder(val_images, "./data/src/val/images", mode="val")
move_files_to_folder(val_labels, "./data/src/val/labels", mode="val")

## テストセットの作成

In [5]:
# 前処理を実行して変換後の画像とアノテーションを保存
#preprocess("./data/src/train", OUT_DIR, IMG_SIZE, OVERRIDE)
#preprocess("./data/src/val", OUT_DIR, IMG_SIZE, OVERRIDE) 
#preprocess("./data/src/test", OUT_DIR, IMG_SIZE, OVERRIDE) 

# YOLO関連ファイルの作成
make_yolo_files(OUT_DIR, mode="train")
# make_yolo_files(OUT_DIR, mode="test")

## モデルの学習
・設定ファイル
　E:\Tomonari\PycharmProjects\DL_Models\venv\Lib\site-packages\ultralytics\cfg
・Albumentations
　E:\Tomonari\PycharmProjects\DL_Models\venv\Lib\site-packages\ultralytics\data

In [1]:
from ultralytics import RTDETR, YOLO

# 利用モデル
# MODEL_NAME = "rtdetr-l"
# model = RTDETR(f"{MODEL_NAME}.pt")
MODEL_NAME = "yolov8m"
#MODEL_NAME = "yolov5lu"
model = YOLO(f"{MODEL_NAME}.pt")

# 学習
model.train(
    data=f"./data/yolo_640/dataset.yaml",
    epochs=200,
    patience=100,
    batch=16,
    imgsz=640,
    cache=True,  # RAM
    device=0,  # CUDA
    name=f"exp-{MODEL_NAME}-ns_ep200",  # experiment name (e.g. "exp-rtdetr-l")
    exist_ok=True,  # nameで指定したフォルダが存在した場合に上書き
    pretrained=True,  # use pre-trained model
    optimizer="auto",  # Optimizer [SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
    close_mosaic=10,  # mixup/mosaicを無効化する最後のエポック数
    mixup=1.0,  # mixupの適用
    dropout=0.0,  # dropout
    scale=0.0,  # scale augmentationは適用しない
    flipud=0.5,  # flipud augmentationの適用
    mosaic=0.1,  # mosaic augmentationは適用しない
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
)

e:\Tomonari\PycharmProjects\DL_Models\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
New https://pypi.org/project/ultralytics/8.0.213 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.5 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./data/yolo_640/dataset.yaml, epochs=200, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=exp-yolov8m-ns_ep200, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_r

## 学習済みモデルを使って推論

In [1]:
from ultralytics import YOLO

# 学習済みモデルのロード
model = YOLO("C:\\Users\\tomonari\\Documents\\SONY_DICE\\data\\runs\\detect\\exp-yolov8m-ns_ep200\\weights\\best.pt")

# 推論の実行
results = model.predict(
    name="predict_yolov8-ns_ep200",
    exist_ok=True,
    source="C:\\Users\\tomonari\\Documents\\SONY_DICE\\data\\yolo_640\\test.txt",
    conf=0.4,
    iou=0.7,
    imgsz=640,
    device=0,  # CDUA
    save=True,  # 推論結果の画像も保存する場合はTrue
    save_txt=True,
    save_conf=True,
    max_det=300,
    agnostic_nms=False,
    classes=[0, 1, 2, 3, 4, 5],
    boxes=True,
)

e:\Tomonari\PycharmProjects\DL_Models\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/24922 C:\Users\tomonari\Documents\SONY_DICE\data\yolo_640\images\test_image0.png: 640x640 2 3s, 98.1ms
image 2/24922 C:\Users\tomonari\Documents\SONY_DICE\data\yolo_640\images\test_image1

## 推論結果の集計（後処理）

In [3]:
import re

def extract_numbers(filename):
    return re.findall(r'\d+', filename)[0] if re.findall(r'\d+', filename) else None

In [4]:
from mylib import load_predicts
from natsort import natsorted

# 推論した結果をDataFrame形式で読み込む
df = load_predicts(r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8-ns_ep200\labels")


In [5]:
# ファイル名から画像の番号を抽出
df["number"] = df['filename'].apply(extract_numbers).astype(int)
# ダイスの目をclassから抽出して、score列を追加
df["score"] = df["class"] + 1

# ダイスの目の合計値を計算
df_score = df.groupby("number")["score"].sum().reset_index().copy()
df_score.sort_values(by="number", inplace=True)  # 'number'列でソート
df_score.head(20)

,number,score
0,0,6
1,1,8
2,2,8
3,3,7
4,4,6
5,5,5
6,6,6
7,7,8
8,8,6
9,9,6


## 提出用ファイルの作成

In [6]:
import os
import pandas as pd

# 結果をタブ区切りでtsvファイルに保存（提出ファイルの形式で保存）
# ファイル名も推論時の .txt から .jpeg に変更する
# ✅ 提出ファイルの保存先やファイル名は、適宜変更してください
OUTPUT_DIR = r"C:\Users\tomonari\Documents\SONY_DICE"

# sample_submitをベースに値を更新して提出ファイルを作成する
submit_sample = pd.read_csv(os.path.join(OUTPUT_DIR, "sample_submit.csv"), header=None, names=["number", "score"])
# submit_sample(DataFrame)の2列目を0埋めする
submit_sample["score"] = 0

# 提出ファイルと予測結果を外部結合して、NaN（欠損値）を0に置き換え
df_submit = pd.merge(submit_sample, df_score, on="number", how="outer")
df_submit.fillna(0, inplace=True)

# 2つのDataFrameのscoreを合算して、新規にscore列を作成（int型に変換する）
df_submit['score'] = df_submit.iloc[:, 1:3].sum(axis=1).astype(int)
df_submit.drop(columns=['score_x', 'score_y'], inplace=True)

# 提出ファイル（CSV）にして保存
df_submit.to_csv(os.path.join(OUTPUT_DIR, "my_submit-ns_ep200.csv"), index=False, header=False)

not_detect = len(submit_sample)- len(df_score)
print(f"検出できなかった画像数: {not_detect}")

検出できなかった画像数: 5


## ダイスを検出できなかった画像の格納（分析用）

In [8]:
import shutil
from mylib import move_files_to_folder
from natsort import natsorted
# 検出出来なかったNumberを抽出し、test_imagexxx.pngにリネーム
not_detect_numbers = set(submit_sample["number"]) - set(df_score["number"])
not_detect_numbers = natsorted(not_detect_numbers)

# 検出できなかった画像をコピーして格納

if not os.path.exists("./data/runs/detect/predict_yolov8-ns_ep200/not_detect"):
    os.makedirs("./data/runs/detect/predict_yolov8-ns_ep200/not_detect")

for number in not_detect_numbers:
    src_file = f"./data/runs/detect/predict_yolov8-ns_ep200/test_image{number}.png"
    dst_file = f"./data/runs/detect/predict_yolov8-ns_ep200/not_detect/test_image{number}.png"
    shutil.copy(src_file, dst_file)

In [17]:
import cv2
import os
import shutil
import numpy as np
from mylib import enum_files

# 標準偏差が85以上の画像を抽出
predict_files =enum_files("./data/yolo_640/images", "test*.png")

if not os.path.exists("./data/runs/detect/predict_yolov8-ns_ep200/error"):
    os.makedirs("./data/runs/detect/predict_yolov8-ns_ep200/error")

for file in predict_files:
    img = cv2.imread(file)
    std = np.std(img)
    if std >=85:
        src_file = f"./data/runs/detect/predict_yolov8-ns/{os.path.basename(file)}"
        dst_file = f"./data/runs/detect/predict_yolov8-ns/error/{os.path.basename(file)}"
        shutil.copy(src_file, dst_file)


KeyboardInterrupt: 

## 精度改善）ダイス×1個の画像抽出

In [5]:
from mylib import load_predicts

# 推論した結果をDataFrame形式で読み込む
# 各バウンディングボックスのサイズを比較するための「w/m, h/m, wh/m」なども計算して追加しています
# ✅ 推論結果を読み込むパスは、環境に合わせて変更してください。
df = load_predicts(r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image\labels")


In [11]:
# ダイスの目をclassから抽出して、score列を追加
df["score"] = df["class"] + 1
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 1
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  18000 non-null  object 
 1   class     18000 non-null  int64  
 2   x         18000 non-null  float64
 3   y         18000 non-null  float64
 4   w         18000 non-null  float64
 5   h         18000 non-null  float64
 6   conf      18000 non-null  float64
 7   score     18000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 1.2+ MB


In [12]:
df["score"].value_counts()

6    3654
4    3630
3    3594
2    3581
5    3541
Name: score, dtype: int64

In [14]:
# ダイスが2個の画像はfilenameが重複するため、これを削除してダイス1個の画像一覧を作成する。
df_one = df.copy()
df_one = df_one.drop_duplicates(subset=["filename"], keep=False)
df_one.head(30)

,filename,class,x,y,w,h,conf,score
0,train_image0.txt,2,0.321256,0.392088,0.547718,0.520076,0.878763,3
0,train_image1.txt,1,0.621021,0.313761,0.573292,0.577791,0.870561,2
0,train_image10.txt,2,0.422524,0.294080,0.548769,0.523275,0.877041,3
0,train_image100.txt,3,0.635577,0.372289,0.570340,0.552973,0.761062,4
0,train_image1000.txt,2,0.672432,0.574861,0.547947,0.555620,0.898946,3
0,train_image10000.txt,3,0.744543,0.673764,0.500954,0.551484,0.880516,4
0,train_image10001.txt,5,0.364280,0.602723,0.527264,0.498778,0.875058,6
0,train_image10004.txt,5,0.264072,0.745022,0.442717,0.483955,0.857747,6
0,train_image10005.txt,4,0.423885,0.253918,0.550757,0.496514,0.886263,5
0,train_image10006.txt,1,0.515405,0.302819,0.569245,0.598258,0.845201,2


In [15]:
df_one["score"].value_counts()

6    1642
5    1635
3    1609
4    1596
2    1544
Name: score, dtype: int64

In [39]:
# 各出目のダイス1個画像をランダムに100枚ピックアップする。
# ピックアップした画像のアノテーションファイルをコピーして、別のフォルダに保存する。
import os
from mylib import move_files_to_folder

SRC_DIR = r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image\labels"
OUTANNO_DIR = r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image\labels_one"

picked_files = df_one.groupby('score').apply(lambda x: x.sample(n=100)).reset_index(drop=True)
picked_files["score"].value_counts()
picked_filenemes = picked_files["filename"].tolist()

anno_src_files = []

for picked_fileneme in picked_filenemes:
    anno_src_file = os.path.join(SRC_DIR, os.path.basename(picked_fileneme))
    anno_src_files.append(anno_src_file)

# 作成したファイル一覧を使って、アノテーションファイルをコピーする。
move_files_to_folder(anno_src_files, OUTANNO_DIR)

# train/valに分割したファイル名一覧のテキストファイルを作成する。
train_files = picked_files.groupby('score').apply(lambda x: x.sample(n=80)).reset_index(drop=True)["filename"].tolist()
# train_files で抽出したファイル以外を val_files に格納する
val_files = picked_files[~picked_files['filename'].isin(train_files)]['filename'].tolist()

# train_files と val_files のファイル名の拡張子を.txtから.pngに変更し、同時に"./image/"を付与する
train_files = [os.path.join("./images/", x.replace(".txt", ".png")) for x in train_files]
val_files = [os.path.join("./images/", x.replace(".txt", ".png")) for x in val_files]

# ファイル名一覧をテキストファイルに保存する。
with open(os.path.join(OUTANNO_DIR, "train.txt"), mode='w') as f:
    f.write('\n'.join(train_files))

with open(os.path.join(OUTANNO_DIR, "val.txt"), mode='w') as f:
    f.write('\n'.join(val_files))


#annotationファイルのデータ修正用緊急プログラム（通常は使用しない）
import pandas as pd
import os
ANNO_FILE = r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image\labels_one"

for anno_file in os.listdir(ANNO_FILE):
    data = pd.read_csv(os.path.join(ANNO_FILE, anno_file), sep=" ", header=None, names=["class", "x", "y", "w", "h", "pred"])
    data.drop(columns=['pred'], inplace=True)
    data.to_csv(os.path.join(ANNO_FILE, anno_file), sep=" ", header=False, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489 entries, 0 to 488
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   class   489 non-null    int64  
 1   x       489 non-null    float64
 2   y       489 non-null    float64
 3   w       489 non-null    float64
 4   h       489 non-null    float64
 5   pred    489 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 23.0 KB


## 追加テストセットの画像確認

In [10]:
import pandas as pd
import os
from mylib import move_files_to_folder
SRC_DIR = r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image"
DST_DIR = r"C:\Users\tomonari\Documents\SONY_DICE\data\yolo_640\images"
COPY_DIR = r"C:\Users\tomonari\Documents\SONY_DICE\data\runs\detect\predict_yolov8_train_image\check_dice_val"

# 画像ファイルの一覧を取得
check_files = pd.read_csv(os.path.join(SRC_DIR, "val_two_for_six.txt"), header=None, names=["filename"])
check_files["filename"] = check_files["filename"].str.replace("./images/", "")

copy_files = []
for chk in check_files["filename"]:
    dst_file = os.path.join(DST_DIR, chk)
    copy_files.append(dst_file)

move_files_to_folder(copy_files, COPY_DIR)
len(copy_files)

e:\Tomonari\PycharmProjects\DL_Models\venv\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


100